In [1]:
import os
import sys


# Insert local paths at the beginning of sys.path
sys.path.insert(0, os.path.abspath('/workspace/nemo/NeMo-opensource/'))

import nemo.collections.asr as nemo_asr
print(nemo_asr.__file__)
import nemo.core as nemo_core
print(nemo_core.__file__)
from nemo.core import adapter_mixins
import nemo
print(nemo.__file__)
import lightning
print(lightning.__file__)
# Restore the site-packages paths
# sys.path.extend(site_packages_paths)

import torch
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
wandb_logger = WandbLogger(project="NEMO_TEST")
# import nemo.collections.asr as nemo_asr

/workspace/nemo/NeMo-opensource/nemo/collections/asr/__init__.py
/workspace/nemo/NeMo-opensource/nemo/core/__init__.py
/workspace/nemo/NeMo-opensource/nemo/__init__.py
/usr/local/lib/python3.10/dist-packages/lightning/__init__.py


In [2]:
TRAIN_MANIFEST = "/disk1/it1/annotations/manifest_train.json"
TEST_MANIFEST = "/disk1/it1/annotations/manifest_train.json"
override_config_file_path = "/workspace/nemo/NeMo-opensource/balu_codes/ctc_model_QuartzNet15x5Base copy.yaml"
conf = OmegaConf.load(override_config_file_path)
OmegaConf.set_struct(conf, True)
model = nemo_asr.models.AV_EncDecCTCModel(conf)

[NeMo I 2024-07-08 23:55:14 collections:317] Dataset loaded with 18351 files totalling 50.98 hours
[NeMo I 2024-07-08 23:55:14 collections:319] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-08 23:55:14 collections:317] Dataset loaded with 18351 files totalling 50.98 hours
[NeMo I 2024-07-08 23:55:14 collections:319] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-08 23:55:15 collections:317] Dataset loaded with 18351 files totalling 50.98 hours
[NeMo I 2024-07-08 23:55:15 collections:319] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-08 23:55:15 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_2.0.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2024-07-08 23:55:15 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_2.0.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2024-07-08 23:55:15 common:815] Instantiating model from pre-trained checkpo

In [3]:
with open_dict(model.cfg):
  # Train Dataloader
  model.cfg.train_ds.manifest_filepath = TRAIN_MANIFEST
  model.cfg.train_ds.batch_size = 32
  model.cfg.train_ds.is_tarred = False
  model.cfg.train_ds.tarred_audio_filepaths = None

  model.cfg.validation_ds.manifest_filepath = TEST_MANIFEST
  model.cfg.validation_ds.batch_size = 32

model.setup_training_data(model.cfg.train_ds)

[NeMo I 2024-07-08 23:55:18 collections:317] Dataset loaded with 18351 files totalling 50.98 hours
[NeMo I 2024-07-08 23:55:18 collections:319] 0 files were filtered totalling 0.00 hours


In [4]:
# model.summarize()
model.freeze()
# model.summarize()
modules_to_train = [model.a_linear, model.v_linear, model.av_encoder, model.av_enocder_layer, model.a_modal_embs
                    , model.v_modal_embs, model.decoder]
for module in modules_to_train:
    module.train()
    for param in module.parameters():
        param.requires_grad = True
    

# model.summarize()

In [5]:
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
max_steps = 3000

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=5, check_val_every_n_epoch=3)

model.set_trainer(trainer)

# Prepare NeMo's Experiment manager to handle checkpoint saving and logging for us
from nemo.utils import exp_manager


# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'test_experiments/',
    name=f"test_wpe_quartz",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
    create_wandb_logger=True,
    wandb_logger_kwargs=OmegaConf.create({"project": "NEMO_TEST", "name": "test_wpe_quartz", "log_model":"all"}),
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

INFO: GPU available: True (cuda), used: True
I0708 23:55:19.239430 134716664817472 rank_zero.py:64] GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
I0708 23:55:19.266383 134716664817472 rank_zero.py:64] TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
I0708 23:55:19.267290 134716664817472 rank_zero.py:64] HPU available: False, using: 0 HPUs


[NeMo I 2024-07-08 23:55:19 exp_manager:396] Experiments will be logged at test_experiments/test_wpe_quartz/2024-07-08_23-55-19
[NeMo I 2024-07-08 23:55:19 exp_manager:856] TensorboardLogger has been set up
[NeMo I 2024-07-08 23:55:19 exp_manager:871] WandBLogger has been set up


[NeMo W 2024-07-08 23:55:19 exp_manager:966] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 3000. Please ensure that max_steps will run for at least 3 epochs to ensure that checkpointing will not error out.


In [6]:
trainer.fit(model)

E0708 23:55:20.059351 134716664817472 jupyter.py:224] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lakshmipathi-balaji. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-07-08 23:55:21 modelPT:767] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )
[NeMo I 2024-07-08 23:55:21 lr_scheduler:772] Scheduler not initialized as no `sched` config supplied to setup_optimizer()



  | Name             | Type                    | Params | Mode 
---------------------------------------------------------------------
0 | a_model          | EncDecCTCModel          | 18.9 M | eval 
1 | a_linear         | Linear                  | 524 K  | train
2 | v_linear         | Linear                  | 262 K  | train
3 | av_enocder_layer | TransformerEncoderLayer | 3.2 M  | train
4 | av_encoder       | TransformerEncoder      | 6.3 M  | train
5 | a_modal_embs     | Embedding               | 512    | train
6 | v_modal_embs     | Embedding               | 512    | train
7 | decoder          | ConvASRDecoder          | 14.9 K | train
8 | loss             | CTCLoss                 | 0      | eval 
9 | wer              | WER                     | 0      | eval 
---------------------------------------------------------------------
10.3 M    Trainable params
18.9 M    Non-trainable params
29.2 M    Total params
116.740   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

[NeMo I 2024-07-08 23:55:22 preemption:56] Preemption requires torch distributed to be initialized, disabling preemption


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 2, global step 1722: 'val_wer' reached 0.26906 (best 0.26906), saving model to '/workspace/nemo/NeMo-opensource/balu_codes/test_experiments/test_wpe_quartz/2024-07-08_23-55-19/checkpoints/test_wpe_quartz--val_wer=0.2691-epoch=2.ckpt' as top 3
I0709 00:11:43.277340 134716664817472 rank_zero.py:64] Epoch 2, global step 1722: 'val_wer' reached 0.26906 (best 0.26906), saving model to '/workspace/nemo/NeMo-opensource/balu_codes/test_experiments/test_wpe_quartz/2024-07-08_23-55-19/checkpoints/test_wpe_quartz--val_wer=0.2691-epoch=2.ckpt' as top 3


[NeMo I 2024-07-09 00:11:50 nemo_model_checkpoint:217] New best .nemo model saved to: /workspace/nemo/NeMo-opensource/balu_codes/test_experiments/test_wpe_quartz/2024-07-08_23-55-19/checkpoints/test_wpe_quartz.nemo


INFO: `Trainer.fit` stopped: `max_steps=3000` reached.
I0709 00:18:02.561257 134716664817472 rank_zero.py:64] `Trainer.fit` stopped: `max_steps=3000` reached.
INFO: Restoring states from the checkpoint path at /workspace/nemo/NeMo-opensource/balu_codes/test_experiments/test_wpe_quartz/2024-07-08_23-55-19/checkpoints/test_wpe_quartz--val_wer=0.2691-epoch=2.ckpt
I0709 00:18:04.404496 134716664817472 rank_zero.py:64] Restoring states from the checkpoint path at /workspace/nemo/NeMo-opensource/balu_codes/test_experiments/test_wpe_quartz/2024-07-08_23-55-19/checkpoints/test_wpe_quartz--val_wer=0.2691-epoch=2.ckpt
INFO: Restored all states from the checkpoint at /workspace/nemo/NeMo-opensource/balu_codes/test_experiments/test_wpe_quartz/2024-07-08_23-55-19/checkpoints/test_wpe_quartz--val_wer=0.2691-epoch=2.ckpt
I0709 00:18:04.564626 134716664817472 rank_zero.py:64] Restored all states from the checkpoint at /workspace/nemo/NeMo-opensource/balu_codes/test_experiments/test_wpe_quartz/2024-07-